<a href="https://colab.research.google.com/github/ol3gka/OTUS_Machine-Learning.-Advanced_2023/blob/main/hw_9_Teach%20me%20senpai/RL_Taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement learning на примере среды Taxi
Q-Learning алгоритм

In [29]:
!pip install gymnasium

import random
import numpy as np
from tqdm import tqdm
import gymnasium as gym

In [ ]:
def epsilon_greedy_policy(Q, state, epsilon):
  """
  if random number > greater than epsilon --> exploitation; # else --> exploration
  """
  if(random.uniform(0,1) > epsilon):
    return np.argmax(Q[state])
  else:
    return env.action_space.sample()

environment:
- 5x5 grid
- Taxi spawned randomly in a square.
- The passenger is spawned randomly in one of the 4 possible locations (R, B, G, Y) and wishes to go in one of the 4 possibles locations

reward system:
- -1 for each timestep
- +20 for successfully deliver the passenger
- -10 for illegal actions (pickup or putdown the passenger at the outside of the destination).

In [7]:
# инициализируем среду
env = gym.make("Taxi-v3", render_mode='ansi')

env.reset()
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [36]:
# инициализируем Q-table
state_space, action_space = env.observation_space.n, env.action_space.n
print(f"Кол-во возможных\n состояний: {state_space} \n действий: {action_space}")
Q = np.zeros((state_space, action_space))
print(Q, Q.shape)

# инициализируем макропараметры
total_episodes = int(5e4)
total_test_episodes = int(1e4)
max_steps = 200
learning_rate = 0.01
gamma = 0.99                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.001           # Minimum exploration probability
decay_rate = 0.01             # Exponential decay rate for exploration prob

Кол-во возможных
 состояний: 500 
 действий: 6
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]] (500, 6)


In [37]:
 # обучаем агента
for episode in tqdm(range(total_episodes)):
    state, step, done = env.reset()[0], 0, False
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    for step in range(max_steps):
        action = epsilon_greedy_policy(Q, state, epsilon)
        new_state, reward, done, _, _ = env.step(action)
        Q[state][action] = Q[state][action] + learning_rate * (reward + gamma *
                                    np.max(Q[new_state]) - Q[state][action])
        if done == True:
            break
        state = new_state

100%|██████████| 50000/50000 [01:08<00:00, 726.88it/s] 


In [46]:
# прогоняем тестовые шаги и визуализируем выйгрышную стратегию
rewards, frames, n_wins = [], [], 0
for episode in tqdm(range(total_test_episodes)):
    state, step, done, total_rewards  = env.reset()[0], 0, False, 0
    for step in range(max_steps):
        if episode == total_test_episodes - 1:
            print(env.render())
        action = np.argmax(Q[state][:])
        new_state, reward, done, _, _ = env.step(action)
        total_rewards += reward
        if done:
            n_wins += 1
            rewards.append(total_rewards)
            break
        state = new_state
env.close()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 10000/10000 [00:05<00:00, 1960.06it/s]

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (North)

+---------+
|R: | : :G|
| : | : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

+---------+
|R: | : :G|
| : | : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

+---------+
|R: | : :G|
| : | : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (East)

+---------+
|R: | : :G|
| : | : : |
| : : : :_|
| | : | : |
|Y| : |B: |
+---------+
  (East)

+---------+
|R: | : :G|
| : | : :_|
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y

In [47]:
# оценка оптимальной стратегии на тестовых иттерациях
print (f"средняя награда за все игры: {str(sum(rewards)/total_test_episodes)}\nколичество побед: {n_wins}")

средняя награда за все игры: 7.9407
количество побед: 10000
